# This Week

* Spatial data manipulation (GeoPandas, Shapely)
* Spatial data analysis (PySAL)

The title of this course is "GIS Programming: Principles of Programming for GIScientists." We have spent most of our time so far focused on the "principles of programming" aspect of the course.  The goal has been to provide you with a solid background in programming for data analysis so that you can use __any__ python library, not just those in the GIS sphere.

This week we (finally) introduce some packages specifically targeted for spatial data. These will be supplemented in the coming weeks when you all present on other specialized packages in your projects.

# Installation 

One of the strengths of Anaconda Python is the ability to create "environments." These are encapsulated python installations that can live side-by-side on your hard drive. This is useful because what you install in one environment does not affect other environments. If one package conflicts with another, you can keep them in their own environments. So far in the semester we have been using the `default` (or `base`) environment. 

Open source spatial packages are notoriously difficult to install. Therefore, we will create a new environment called `gis2019` that has the basic spatial tools you need to know. 

Open up a new Anaconda Prompt (Windows) or Terminal (Mac) and type the following (press `Enter` after each line): 
```
conda create --name gis2019 -c conda-forge python=3 geopandas jupyter seaborn palettable spyder ipykernel
conda activate gis2019
ipython kernel install --user
```
>If you are running this in Windows, enter the line below.
>```
conda install pywin32
>```

Running the above lines will take awhile to install. **Notice we are installing Python3 in this environment.** Anaconda is flexible in this way. GeoPandas and its dependencies are not compatible with Python2.

#### After installing the environment

* Once it is done, close and re-open Jupyter Notebook and return to this notebook. 
* Then click `Kernel` > `Change kernel` > `Python 3`
* Assuming all went well, you should be ready to go with geopandas.

In [ ]:
import geopandas as gpd
gpd.__version__

In [ ]:
import shapely
shapely.__version__

__STOP__: If you don't have geopandas version 0.6.1 and shapely version at least 1.6.4.post1, post to the [associated Canvas discussion](https://canvas.fsu.edu/courses/102434/discussion_topics/589658) what you do have.

# GeoPandas

>GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting.

What GeoPandas does...

* Geometry operations (Shapely)
* Data alignment (pandas)
* Coordinate transformations (pyproj)
* Read/write GIS file formats (Fiona)
    * Shapefiles
    * PostGIS
    * geoJSON
* Plotting (matplotlib)

GeoPandas has many dependencies. You already have pandas and matplotlib installed, and now you will get:

* Shapely 
* Fiona
* Pyproj

The documentation on geopandas is thin relative to other packages you've seen this semester. However, geopandas is tightly coupled with its dependencies, so it is often the case that help resources are better found in the documentation for [shapely](http://toblerity.org/shapely/) or [pandas](http://pandas.pydata.org/).

### Data Structures

Geopandas is built around the **GeoDataFrame**. The key difference between the pandas (DataFrame) and geopandas (GeoDataFrame) versions is the **`geometry` column**, which contains the spatial information on the observation. If you had a DataFrame with columns `tree_height` and `tree_diameter`, a GeoDataFrame would have three columns: `tree_height`, `tree_diameter` and `geometry`. In this case the `geometry` column would hold the geographic coordinates of the tree locations. The `geometry` column can hold point, line or polygon coordinates.

You can kinda think of the `geometry` column containing the `.shp` part of a shapefile and the other columns being the stuff in the `.dbf` part of a shapefile. The GeoDataFrame mashes these two parts into one _object_. This clear link between the data and coordinates is very powerful and quite intuitive, IMHO. If you delete or select a row (or rows) in the GeoDataFrame using standard pandas slicing, the spatial part just comes along for the ride since it's another column. 

We did not talk much about the **Series** object in pandas; it is like a single column from a pandas DataFrame. The counterpart in GeoPandas is the **GeoSeries**. Whereas a Series object can contain any content that is legit for a DataFrame column, a GeoSeries is exclusively a `geometry` column. It is like a shapefile without the `.dbf` part.

### Reading and manipulating spatial data

Reading in a shapefile is similar to how we read in a CSV in pandas.

In [ ]:
fl_counties = gpd.read_file('spatial_data/cntbnd_jul11.shp')

...but now we get a GeoDataFrame instead of a DataFrame.

In [ ]:
type(fl_counties)

Since the GeoDataFrame is built on top of pandas, we get lots of our familiar tools.

In [ ]:
fl_counties.head()

**Action**: Open `cntbnd_jul11.shp` in ArcGIS or QGIS. Notice that the columns in the GeoDataFrame above match what is in the attribute table, with the exception of the `geometry` column.

The result of the `shape` attribute below indicates that we have all 67 counties in the state of Florida.

In [ ]:
fl_counties.shape

Might be good to take a peek at a map to confirm that we've got them all.

In [ ]:
# This cell has some boilerplate plotting setup
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 8,8  # change this line if you want to change the default map size

The following line may be a little slow.

In [ ]:
# One-liner to get a basic map
fl_counties.plot()

Let's stop for a second and absorb what just happened. We opened a shapefile with one line:
```
fl_counties = gpd.read_file('spatial_data/cntbnd_jul11.shp') 
```
and then plotted it with one line:
```
fl_counties.plot()
```
Not only that, we have access to all the attributes in the DBF.

In [ ]:
fl_counties.columns

In [ ]:
fl_counties.NAME

But what's going with the `geometry` column?  Let's take a look at the first county in the data frame.

In [ ]:
type(fl_counties.geometry[0])

So its type is a shapely `Polygon`. Again we see that geopandas relies heavily on its dependencies.  What is inside of a shapely `Polygon`?

In [ ]:
print(fl_counties.geometry[0])

Whoa!!! It contains all the X-Y coordinates needed to draw the polygon. Remember from introductory GIS that a polygon is represented by series points, and the GIS then "connects the dots" to form a polygon.

In the next cell we rerun the previous cell without the `print` statement. Geopandas has some nice defaults, eh?

In [ ]:
fl_counties.geometry[0]

In [ ]:
cty_idx = 5
print(fl_counties.NAME[cty_idx])
fl_counties.geometry[cty_idx]

Let's make a new object with just Leon County, the same way we would do with pandas.

In [ ]:
leon = fl_counties.loc[fl_counties.NAME=='LEON',:]
leon.shape

In [ ]:
leon.plot()

**Note**: The new object is also a GeoDataFrame with the same 8 columns we had before, and the same methods and attributes; but only one row. 

Raise your hand if you thought the `leon` object was a GeoSeries. It is not; it is still a GeoDataFrame. Why? A GeoSeries is a single _column_ (just the `geometry` part); in contrast, the `leon` object is a single _row_ with multiple columns.

(Don't worry, no one was looking if you got that question wrong.) 

Now let's read in the major roads for Florida.

In [ ]:
fl_roads = gpd.read_file('spatial_data/majrds_oct15.shp')
fl_roads.shape

In [ ]:
fl_roads.head()

In [ ]:
fl_roads.plot()  # this is slow

Let's look at the `geometry` column for the roads file. The county data frame contained shapely `Polygon` objects, what type do you think the roads will be?

In [ ]:
type(fl_roads.geometry[0])

In [ ]:
print(fl_roads.geometry[0])

That is a pretty big file, which takes a long time to draw. Let's clip it to just Leon County. (It will take a couple of steps to do this.)

We'll look at two approaches to getting just the Leon County roads. First, we will find the intersection of the Leon County polygon and the Florida roads lines. (This is slow.)

In [ ]:
leon_shp = leon.geometry.iloc[0]  # we are isolating just the Leon county polygon
type(leon_shp)

The next line is slow.

In [ ]:
leon_roads1 = fl_roads.intersection(leon_shp)

__Note__: Notice that the `intersection` method takes a shapely object NOT a geopandas object. This is important to remember.

In [ ]:
leon_roads1.plot()

Second, we'll find all the roads that `intersect` Leon County.

In [ ]:
fl_roads.intersects(leon_shp).head()

In [ ]:
leon_roads2 = fl_roads.loc[fl_roads.intersects(leon_shp),:]

In [ ]:
leon_roads2.plot()

Hmmmmm. Something looks fishy here. The first plot looks pretty good, right? The second plot looks kinda like the roads in Leon County, but there is some extra stuff hanging around. What's happening? The difference lies in `intersection` vs. `intersects`.

Recall that `intersection` took a lot longer to run than `intersects`? If you don't remember, rerun the two approaches. 

* `intersects` is a _question_: "Does any part of this road lie in (i.e., intersects) Leon County?" It returns a boolean (`True` or `False`) for each road.
* `intersection` is an _action_: "Use the Leon County polygon as a cookie cutter on the Florida roads." It returns geometries, i.e., line segments.

But the story doesn't end there. Neither `leon_roads1` nor `leon_roads2` is perfect alone.  Let's see why.

In [ ]:
leon_roads1.head()

In [ ]:
print(leon_roads1.shape)
print(fl_roads.shape)

**Note**: `leon_roads1` is a GeoSeries of the same length as `fl_roads`. It essentially zeros out the roads that don't intersect the polygon; thus keeping just the intersection of the roads that do intersect the polygon. Two issues here:

  1. A GeoSeries only contains geometries, not all the attributes.
  2. We don't really need to keep all the records for zeroed out rows.

Let's do the same with `leon_roads2`.

In [ ]:
leon_roads2.head()

In [ ]:
print(leon_roads2.shape)
print(fl_roads.shape)

**Note**: `leon_roads2` has the shapefile attributes we want and is the correct length. However, it has those extra bits of road that fall outside Leon County.

I know that you are sitting on the edge of your seat wondering... How will he resolve this? Will he resolve this? Will this be a quiz question?

In [ ]:
fl_roads_temp = fl_roads.copy()
fl_roads_temp.geometry = fl_roads_temp.intersection(leon_shp).geometry
leon_roads = fl_roads_temp.loc[fl_roads_temp.intersects(leon_shp),:]

Looking above...
1. Make a copy of the state roads object.
2. Clip the state roads to Leon County, but only save the geometries. Recall that the problem here is that we have a lot of rows with zeroed out geometries.
3. Select just those roads that fall within Leon County.

In [ ]:
leon_roads.plot()

In [ ]:
leon_roads.shape

In [ ]:
leon_roads.head()

**Action**: The process of selecting the Leon County roads only takes three lines of python code. It is simple, but not as intuitive as we've come to expect from python. This was my best attempt for solving the problem. Can you identify a simpler (or more intuitive) way of getting this Leon County roads GeoDataFrame?  If you've got another approach, post it to the discussion board.

Let's bring it all together in a single plot.

In [ ]:
ax = leon.plot(color='green')
leon_roads.plot(ax=ax, color='blue')

How did that plot work? When you want to plot two things on top of each other:
- Plot the first layer as before, but now assign it to a variable
- Plot the second layer as before, but now pass the first layer to the `ax` argument in the second

Let's plot some points on our map. We will open a national Twitter dataset provided by http://www.followthehashtag.com.

In [ ]:
import pandas as pd
tweets_raw = pd.read_csv('spatial_data/dashboard_x_usa_x_filter_nativeretweets.csv',
                         encoding = "latin1")

In [ ]:
tweets_raw.shape

In [ ]:
tweets_raw.columns

This was a plain CSV file, which we opened as a regular pandas dataframe. We now need to convert to a geodataframe. The geogataframe needs shapely objects.  You've already seen polygons and lines, now it's time for points.

In [ ]:
tweets_raw['xy'] = list(zip(tweets_raw.Longitude, tweets_raw.Latitude))
tweets_raw['geometry'] = tweets_raw.xy.apply(shapely.geometry.Point)
tweets = gpd.GeoDataFrame(tweets_raw)

Let's unpack the cell above.

1. We "zip" the two columns together. This takes the content from the two columns and makes a tuple out of them. You can verify this with `tweets_raw.xy.head()`.
2. The `apply` method is a really useful tool for a dataframe. It essentially says, "apply this action to each row in the dataframe." In this case we are saying, "make each element in the `tweets.xy` column a shapely `Point` object."
3. Now that we have all the parts, we can convert the `tweets` dataframe into a geodataframe.

In [ ]:
tweets.head()

Let's take a look at our tweets. Since there are over 200,000 tweets, we want to use some alpha and make each point small. This is a little slow to plot.

In [ ]:
tweets.plot(alpha=0.1, markersize=2)

__Action__: Play around with the marker size and alpha to see how it affects the information being communicated.

**Note**: I'm sure you noticed that there will be a problem placing these points on our Leon County map from earlier.  Hint: check out the scales on the above map and the ones earlier. Okay, since it's easy to do, let's just see what happens if we work with the data as is.

In [ ]:
ax = leon.plot(color='green')
tweets.plot(ax=ax, color='blue')

The [CRS](https://en.wikipedia.org/wiki/Spatial_reference_system) for our roads and counties were read in from the shapefile by geopandas. And these were transferred down the line as we sliced and diced the GeoDataFrames.

In [ ]:
leon_roads.crs

Recall that earlier we learned that the `geometry` column contains the content from the `.shp` part of a shapefile and that the other columns contain the content from the `.dbf` part? Now we have learned that the `crs` attribute contains the information from the `.prj` part of a shapefile.

In this case we are using `epsg:3087`, which is [Florida GDL Albers](http://spatialreference.org/ref/epsg/nad83harn-florida-gdl-albers/).

We built the twitter GeoDataFrame from a CSV, so CRS info was not automatically populated. This is a really important concept to understand. A CSV file does not have any information on the projection of the data, but a shapefile does. 

Seriously, stop for a moment and think about this; it is a surprisingly difficult concept to absorb, but very important that you understand this. 

When pandas reads in the twitter data, it sees the `Longitude` and `Latitude` columns as any other columns containing floats. It has no way of knowing these are special. Guess what, ArcGIS doesn't know either. 

In [ ]:
print(tweets.crs)

Geopandas needs to know the current CRS in order to convert to another CRS. We know that we have latitude and longitude from Twitter, so we can just populate the current CRS directly. Here is a [nice summary of using CRS in R](https://www.nceas.ucsb.edu/~frazier/RSpatialGuides/OverviewCoordinateReferenceSystems.pdf), but as you can see below it is largely the same in Python.

In [ ]:
tweets.crs = {'init': 'epsg:4326'}   

In [ ]:
print(tweets.crs)

The actual conversion is in the next cell. We'll just steal the CRS from the GeoDataFrame we want to match.

In [ ]:
tweets.to_crs(crs=leon_roads.crs, inplace=True)

Why was that last cell kind of slow? It was doing the math on every point to convert from one projection to another.

In [ ]:
print(tweets.crs)

Let's plot our tweets. Notice that they are in the correct projection now.

In [ ]:
tweets.plot(alpha=0.1, markersize=2)

Now we need to select just the Tweets in Leon County. Recall that we did this earlier to select roads in Leon County. 

In [ ]:
tweets_leon = tweets.loc[tweets.intersects(leon_shp),:]

Now for a real GIS question (finally!). Why is isolating the points in Leon County so much easier than isolating the roads in Leon County?

In [ ]:
ax = leon.plot(color='green')
tweets_leon.plot(ax=ax, color='blue', alpha=0.25)

In [ ]:
ax = leon.plot(color='green')
leon_roads.plot(ax=ax, color='red', linewidth=0.5)
tweets_leon.plot(ax=ax, color='blue', alpha=0.5)
ax.set_xlim(360000,390000)
ax.set_ylim(705000,730000)

Notice in the above cell, if we want to fine tune the plot we need to use matplotlib syntax.

The results of your work can then be written out to the hard drive.

In [ ]:
tweets_leon.drop('xy', 1, inplace=True)
tweets_leon.to_file('spatial_data/tweets_leon.shp')
leon_roads.to_file('spatial_data/leon_roads.shp')
leon.to_file('spatial_data/leon_county.shp')

There are lots of other fun things you can do with geopandas.

In [ ]:
fsu_tweets = tweets[tweets['Tweet content'].str.contains('fsu ', case=False)]
for i in fsu_tweets['Tweet content']:
    print(i)
    print('------')

And like all things related to pandas, you can string together crazy one-liners...

In [ ]:
tweets[tweets['Tweet content'].str.contains('fsu ', case=False)].buffer(100000).plot()

# Python Spatial Analysis Library (PySAL)

>PySAL is an open source cross-platform library of spatial analysis functions written in Python. It is intended to support the development of high level applications for spatial analysis. First and foremost, PySAL is a library in the fullest sense of the word. Developers looking for a suite of spatial analytical methods that they can incorporate into application development should feel at home using PySAL. Spatial analysts who may be carrying out research projects requiring customized scripting, extensive simulation analysis, or those seeking to advance the state of the art in spatial analysis should also find PySAL to be a useful foundation for their work.

The examples below are based on Jupyter Notebooks by Dani Arribas-Bel and Serge Rey, which can be found at [github.com/pysal/notebooks](https://github.com/pysal/notebooks).

### Installation

Pysal came along with the geopandas installation. If it did not and the cell below does not work correctly, run `conda install pysal` inside your newly created `gis2019` environment.

In [ ]:
import pysal.lib as ps

In [ ]:
print(ps.__version__)

### Leon County Demographics

We'll look at demographic data on census tracts in Leon County. A census tract is a statistical geography used for aggregating data collected by the US Census Bureau; these polygons tend to have approximately 4,000 people, and are often referred to as "neighborhoods."

### Reading in Files

We'll start by opening the shapefile using the pysal reader.

In [ ]:
shp = ps.io.open('spatial_data/leon_tracts.shp')

Various geometric information is immediately available.

In [ ]:
shp.header

In [ ]:
shp.bbox  # bounding box of the full dataset

You can inspect the individual observations by converting them into a list.

In [ ]:
polys = list(shp)
print("number of tracts in Leon County:", len(polys))

In [ ]:
tract0 = polys[0]
print(tract0)
print(tract0.area)

Notice that pysal has its own polygon type whereas geopandas uses the shapely polygon type.

In [ ]:
tract0.centroid

In [ ]:
print(tract0.vertices)

In pysal, the `.shp` file is handled separately from the `.dbf` file. While the former contains the geometric information, the latter contains the attributes.

In [ ]:
dbf = ps.io.open('spatial_data/leon_tracts.dbf')

**Note**: We opened the `.shp` and `.dbf` files using the same `ps.io.open()` function. Pysal uses the file extension to determine how to open the file.

Similar to the `shp` object above, you can also inspect the `dbf` object. The following cell shows the column headers from the attribute table.

In [ ]:
dbf.header

For those who like databases, you can see the database specifications for each column.

In [ ]:
dbf.field_spec

You can see what is happening in any particular column.

In [ ]:
print(dbf.by_col('ALAND'))

The `by_col` method returns a list. Sometimes it's better to get a numpy array.

In [ ]:
dbf.by_col_array(['ALAND'])

Multiple columns can be extracted at the same time.

In [ ]:
dbf.by_col_array(['ALAND', 'AWATER']).shape

Let's read in some demographic data. It is typical that demographic data is acquired separately from the geometric information; which means they need to be merged after downloading.

In [ ]:
import pandas as pd
csv = pd.read_csv('spatial_data/leon_tract_ests.csv')
csv.shape

In [ ]:
csv.head()

Let's swap out the cryptic names for something more useful.

In [ ]:
csv.columns = ['GEOID', 'population', 
               'pov_base', 'below_pov',
               'edu_base', 'bach_higher',
               'emp_base', 'unemp',
               'households', 'sing_parent_hh']

In [ ]:
csv.head()

We need to be sure that the demographic data and the geometric data are in the same order. The `.dbf` file associated with the `.shp` file has `GEOID` column as does the `.csv` file, however the `.csv` version is prepended with a "g". 

In [ ]:
csv.index = [i[1:] for i in csv.GEOID]
target_index_order = dbf.by_col['GEOID']
csv = csv.reindex(target_index_order)

In [ ]:
dbf.by_col['GEOID'][0:5]

In [ ]:
csv.head()

**Action**: The dbf and csv data are now in the same order. We test this in the following cell. If you don't understand what is happing in the next cell, copy and paste the contents to a new cell and deconstruct it. Hmmmm... deconstructing a complex one-liner might make for a good quiz question. There are a few in this Notebook. You should practice how to do this and how to interpret the various parts.  Can you explain each part in the cell below?

In [ ]:
sum(dbf.by_col_array(['GEOID']).flatten() == csv.index)

### Spatial Weights

At the core of much spatial analysis is the spatial weights matrix ($W$). The $W$ in an $n \times n$ matrix that defines the relationship between all observations on the map. In the case of Leon County the matrix will be $68 \times 68$.  

A common weights structure is queen contiguity. This can be built directly from the shapefile.

In [ ]:
w = ps.weights.Queen.from_shapefile('spatial_data/leon_tracts.shp', idVariable='GEOID')

The `w` object has a lot of attributes and methods.

In [ ]:
print(dir(w))

In [ ]:
w.n  # number of observations

The `neighbors` attribute is a dictionary, where the key is the ID of interest and the value is a list of its neighbors' IDs.

In [ ]:
w.neighbors

**Note**: The weights matrix is stored in what is known as a [sparse](https://en.wikipedia.org/wiki/Sparse_matrix) format. Sparse data structures (of which there are many) only store the non-zero elements of the matrix. Technically there are 68*68 (4,624) elements in this matrix but we only need to store 406 of them since the rest are zero.

In [ ]:
w.nonzero

The average number of neighbors...

In [ ]:
w.mean_neighbors

The distribution of neighbor relationship can be visualized.

In [ ]:
import numpy as np
w_hist_data = np.array(w.histogram)
fig, ax = plt.subplots(figsize=(5,5))
ax.bar(w_hist_data[:,0], w_hist_data[:,1], align='center', width=1)
ax.set_xlabel('number of neighbors')
ax.set_ylabel('number of tracts')

**Note**: Looking at this histogram, you can see for example that there are three census tracts with nine neighbors.

The `weights` attribute is the same structure as `neighbors` except that the value is a list of the weights.

In [ ]:
w.weights

**Note**: Since this is a queen contiguity matrix, all the non-zero weights are one. Pysal also offers rook, bishop and distance weights.

The weights can be row-standardized so that the total weight for any observation sums to one.

In [ ]:
w.transform = 'R'
w.weights

**Action**: Look at the output above and notice that if you sum the weights for any observation, the total is one.

Distance based weights are also available.

In [ ]:
w_dist = ps.weights.Kernel.from_shapefile('spatial_data/leon_tracts.shp', idVariable='GEOID')

In [ ]:
w_dist.mean_neighbors

In [ ]:
w_dist.weights  # this produces a lot of output!

### Spatial Analysis

#### Global Spatial Autocorrelation

Most spatial data are spatially autocorrelated:

First law of geography (W. Tobler):

> Everything is related to everything else, but near things are more related than distant things. 

Another take on this (M. Goodchild):
> Only Hell is spatially random.

We can measure spatial autocorrelation with the Moran's I. Let's see if we live in hell, i.e., identify if Leon County demographics are random or not.

In [ ]:
import pysal.explore as pse
pov_rate = csv.below_pov/(csv.pov_base*1.0)
m_pov = pse.esda.Moran(pov_rate, w)
m_pov.I

**Note**: Positive values of Moran's I indicate positive spatial autocorrelation (i.e., observations tend to be near observations that are similar to themselves), negative values indicate negative spatial autocorrelation (i.e., observations tend to be near observations that are different from themselves) and values near zero indicate a random pattern. The maximum and minimum values for Moran's I vary from dataset to dataset, but the possible values tend to range from about -1 to 1. 

In [ ]:
m_pop = pse.esda.Moran(csv.population, w)
m_pop.I

**Note**: Let's compare these two results. We got a relatively high value for the poverty rate, indicating that poorer neighborhoods tend to be near one another and wealthier ones tend to be near one another. In contrast, the Moran's I value is close to zero for the population variable, seeming to indicate that there is no (or very little) spatial autocorrelation in the number of people in a census tract.  Does this mean that we are [living in hell](https://youtu.be/ydqkBG22Tk8) as Goodchild hinted? Recall that census tracts are designed to have approximately 4,000 people each so it's not surprising that there is no spatial pattern to these values; in contrast, poverty is the result of myriad societal processes manifest in Tobler's first law of geography.

We can also look at the statistical significance of a Moran's I value. There are a number of ways to compute the significance of Moran's I; we'll consider the p-value based on simulations of Moran's I. 

In [ ]:
print(m_pov.p_sim)
print(m_pop.p_sim)

**Note**: Typical thresholds for identifying if a value is statistically significant are p-values of 0.01, 0.05 and 0.10. The results above indicate that the Moran's I for poverty rate is highly significant (its p-value is far below 0.01). The p-value for population on the other hand is above even the generous threshold of 0.10, indicating that even though the Moran's I is positive (0.051) it is not statistically different from zero (recall that zero indicates spatial randomness).

You can get a feel for the Moran's I results by looking at a scatter plot of the value of interest (in our case poverty rate or population) against the spatial lag of the variable (the average of the value in its neighboring census tracts). 

In [ ]:
fig, ax = plt.subplots(1,2)
w_pov_rate = ps.weights.lag_spatial(w, pov_rate)
#plt.figure(figsize=(5,5))
ax[0].scatter(pov_rate, w_pov_rate, marker='.', s=20, alpha=1, color='k')
ax[0].set_xlim(0,1)
ax[0].set_ylim(0,1)
ax[0].set_aspect('equal')
ax[0].set_title("Poverty Rate")
w_population = ps.weights.lag_spatial(w, csv.population)
ax[1].scatter(csv.population, w_population, marker='.', s=20, alpha=1, color='k')
ax[1].set_xlim(0,10000)
ax[1].set_ylim(0,10000)
ax[1].set_aspect('equal')
ax[1].set_title("Population")

**Note**: There is a clear positive trend between the poverty rate and its spatial lag (indicating positive spatial autocorrelation), but for population the plot does not show much pattern.

The global Moran's I can identify if there the is an overall spatial pattern to the data. Local measures of spatial autocorrelation show if certain parts of the map exhibit spatial patterns in the data. In this case, you get a spatial autocorrelation value for each observation, instead of one for the entire map.

In [ ]:
mloc_pov = pse.esda.Moran_Local(pov_rate.values, w)
mloc_pov.Is

You also get a p-value for each.

In [ ]:
mloc_pov.p_sim

Pysal has some basic plotting functionality, which is useful for visualizing the Local Moran's I. 

In [ ]:
from pysal.viz.splot.esda import lisa_cluster
leontracts = gpd.read_file('spatial_data/leon_tracts.shp')
lisa_cluster(mloc_pov, leontracts, legend_kwds={'loc': 'upper left'})

**Note**: The map shows tracts in red when they contain a relatively high poverty rate and are surrounded by other high tracts.  Blue tracts are relatively low poverty tracts, surrounded by other low poverty tracts. Does this map follow your intuition about the distribution of poverty in Leon County?

### Overview

This was a brief introduction to two python packages focused on spatial data; there are a lot more.  I'm looking forward to seeing your presentations.

# Test Yourself

1a) Open any shapefile you have on your computer using geopandas.

1b) Plot the geodataframe.

2a) Open the Leon County census tracts shapefile using geopandas (you used this in the pysal example).

2b) A geodataframe has a `centroids` attribute, which contains the geographic center of each each polygon. Create a new geodataseries or geodataframe containing the centroids and make one plot with the census tract polygons and census tract centroids.